<a href="https://colab.research.google.com/github/olegkleiman/udemy_nlp/blob/main/Malaria_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [ ]:
dataset, dataset_info = tfds.load('malaria',
          with_info=True,
          as_supervised=True,
          shuffle_files=True,
          split=['train'])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]